In [209]:
import cv2
cv2.__version__

'3.3.1'

In [210]:
import sys
print(sys.version)

3.6.5 | packaged by conda-forge | (default, Apr  6 2018, 16:13:55) [MSC v.1900 64 bit (AMD64)]


In [211]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk
import time
import pandas as pd

import skimage.io as io

import os

import imageio
# print environment
os.path.dirname(sys.executable)

'C:\\Users\\calli\\Anaconda3\\envs\\opencv2'

In [212]:
#vidPath = "D:/BeeTwoFlowerExperiment/2018_02_27__14_04_52_556/2018_02_27__14_04_55_680_cam1-0000.avi"
vidPath = r"D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52_556\2018_02_27__14_04_55_680_cam1-0000.avi"
os.path.isfile(vidPath)

True

In [11]:
cap = cv2.VideoCapture(vidPath)


length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print(length, width, height, fps)
cap.release()

546 1280 1024 10.0


In [12]:
def load_imgs(vidPath, start, stop):
    cap = cv2.VideoCapture(vidPath)
    imgs = []
    for ff in range(length):
        
        
        ret, frame = cap.read()
        
        if np.mod(ff, 10) == 0:
            print(ff)
        

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        imgs.append(img)
        
        if ff > stop:
            break
            
    cap.release()
    return imgs

In [13]:
imgs = load_imgs(vidPath, 0, 50)

0
10
20
30
40
50


In [14]:
io.imshow(imgs[0])

In [15]:
r = cv2.selectROI(imgs[0], showCrosshair = True, fromCenter = False)
cv2.destroyAllWindows()

In [16]:
r

(438, 625, 354, 159)

In [17]:
# possible opencv events
events = [i for i in dir(cv2) if 'EVENT' in i]
print( events )

['EVENT_FLAG_ALTKEY', 'EVENT_FLAG_CTRLKEY', 'EVENT_FLAG_LBUTTON', 'EVENT_FLAG_MBUTTON', 'EVENT_FLAG_RBUTTON', 'EVENT_FLAG_SHIFTKEY', 'EVENT_LBUTTONDBLCLK', 'EVENT_LBUTTONDOWN', 'EVENT_LBUTTONUP', 'EVENT_MBUTTONDBLCLK', 'EVENT_MBUTTONDOWN', 'EVENT_MBUTTONUP', 'EVENT_MOUSEHWHEEL', 'EVENT_MOUSEMOVE', 'EVENT_MOUSEWHEEL', 'EVENT_RBUTTONDBLCLK', 'EVENT_RBUTTONDOWN', 'EVENT_RBUTTONUP']


In [34]:
len(imgs)

52

In [208]:
# mouse hover and draw


vid = imgs



rect = len(imgs) * [[None, None, None, None,None, None, None, None]]
ix,iy = -1,-1
drawing  = False
moving = False
buttonRelease = False
doneFrames = []
# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy, drawing, moving, im3, buttonRelease, xx, yy, im2
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
    elif event == cv2.EVENT_MOUSEMOVE:
        moving = True
        im3 = img
        #cv2.circle(img,(x,y),10,(255,0,0, 0.1),-1)
        cv2.line(img,(img.shape[1],y),(0,y),  color = (0,0,0))
        cv2.line(img,(x,0),(x,img.shape[0]),  color = (0,0,0))
        if drawing == True:
            cv2.rectangle(img,(ix,iy),(x,y),(100,100,255),1)

    elif event == cv2.EVENT_LBUTTONUP:
        buttonRelease = True
        #moving = True
        drawing = False
        cv2.rectangle(im2,(ix,iy),(x,y),(0,255,0),1)
        xx, yy = x,y
        
    
img = imgs[0].copy()
im2 = img.copy()
im3 = img.copy()
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

ctr = 0
behavior = ""

while(1):
    if ctr < 0:
        ctr = 0
        cv2.putText(img,"1st frame of vid",(450,30), font, 1,(100,100,100),2,cv2.LINE_AA)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img,'frame:' + str(ctr),(1,30), font, 1,(100,100,100),2,cv2.LINE_AA)
    cv2.putText(img,'f:forward one frame, no change to data',(1,500), font, 1,(100,100,100),2,cv2.LINE_AA)
    cv2.putText(img,'n:next frame, no bee in this frame',(1,530), font, 1,(100,100,100),2,cv2.LINE_AA)
    
    if ctr in doneFrames:
        cv2.putText(img,'frame:' + str(ctr) + " already done",(1,30), font, 1,(100,100,100),2,cv2.LINE_AA)
    if buttonRelease: 
        cv2.putText(img,"Press a key to indicate behavior",(1,60), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"l:landing",(1,90), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"f:flying",(1,120), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"p:pollinating",(1,150), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"g:grooming (while landed)",(1,180), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"t:takeoff",(1,210), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"a:add manual note (in other window)",(1,240), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"f:forward frame- don't save data",(1,270), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"esc:retry drawing the box",(1,300), font, 1,(100,100,100),2,cv2.LINE_AA)
        cv2.putText(img,"n:next frame, no bee in frame",(1,330), font, 1,(100,100,100),2,cv2.LINE_AA)
        
    cv2.imshow('image',img)
    if moving: # if mouse is moving, then reload the image
        img = im2.copy()
    else:
        img = im3
    moving = False
    if buttonRelease:
        note = ""
#         print([ix, iy, xx, yy])
        buttonRelease = False
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
        while(1):
            k = cv2.waitKey(1) & 0xFF
            if k == ord('l'): # landing
                break
            elif k == ord("p"): # pollinating
                break
            elif k == ord("f"): # flying
                break
            elif k == ord("g"): # grooming
                break
            elif k == ord("t"): # takeoff 
                break
            elif k == ord("a"): # add note
                note = input("Notes ")
                break
            elif k == ord("n"): #move forward, no bee in frame
                ix, iy, xx, yy, note, = np.nan, np.nan, np.nan, np.nan, "no bee in frame"
                break
            elif k == ord('b'): # back one frame
                ctr -= 1
                break
            elif k == ord('f'):  # if there is no bee in frame
                ix, iy, xx, yy = np.nan, np.nan, np.nan, None
                note = ""
                break
            elif k == 27:
                #ctr -= 1
                ix, iy, xx, yy, note, = np.nan, np.nan, np.nan, np.nan, None
                k = ord("e")
                break  
            
        if (ctr >= 0) and( k != ord("e")):         
            rect[ctr] = [ix, iy, xx, yy, int(ctr), vidPath, chr(k), note]
            doneFrames = [ii for ii in range(len(rect)) if None not in rect[ii]]  
            ctr += 1    

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break  
    elif k == ord("b"): # back one frame
        ctr -= 1
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
    elif k == ord("n"): # if there is no bee in frame
        rect[ctr] = [np.nan, np.nan, np.nan, np.nan, int(ctr), vidPath, chr(k), note]
        doneFrames = [ii for ii in range(len(rect)) if None not in rect[ii]] 
        ctr += 1
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
        note = "no bee in frame"
    elif k == ord("f"): # move forward without inputting any data
        ctr += 1
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
        note = "no bee in frame"
        
cv2.destroyAllWindows()

rect2 = [rect[ii] for ii in doneFrames]
pd.DataFrame(rect2, columns = ["x1", "y1", "x2", "y2", "frameNum", "vidPath", "behavior", "notes"])

,x1,y1,x2,y2,frameNum,vidPath,behavior,notes
0,304,429,705,581,0,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,f,
1,433,601,774,823,1,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,


In [197]:
rect

[[None, None, None, None, None, None, None, None],
 [nan,
  nan,
  nan,
  nan,
  1,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  2,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  3,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  4,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  5,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  6,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee i

In [112]:
doneFrames = [ii for ii in range(len(rect)) if None not in rect[ii]]
doneFrames

[0, 1, 2, 3, 5, 6, 7, 8]

In [17]:
[rect[ctr-1][4] == doneFrames[ii] for ii in range(len(doneFrames))]

[False, False, False]

In [80]:
rect2 = [x for x in rect if x is not None]
rect2

[[nan,
  nan,
  nan,
  nan,
  1,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  2,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  3,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  4,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  5,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  6,
  'D:\\BeeTwoFlowerExperiment\\2018_02_27__14_04_52_556\\2018_02_27__14_04_55_680_cam1-0000.avi',
  'n',
  'no bee in frame'],
 [nan,
  nan,
  nan,
  nan,
  7,
  'D:\\

In [19]:
rect[ctr][4] in [rect[i][4] for i in range(len(rect))]

IndexError: list index out of range

In [ ]:
doneFrames = [rect[i][4] for i in range(len(rect))]

whichFrames = [rect[ctr][4] == doneFrames[ii] for ii in range(len(doneFrames))]
any(whichFrames)
doneIndex = [i for i, x in enumerate(whichFrames) if x]
doneIndex

In [ ]:
img = np.zeros((512,512,3), np.uint8)
cv2.rectangle(img,(33 ,28),(496 ,494),(0,255,0),-1)
alpha = 0.1
overlay = img.copy()
cv2.addWeighted(overlay, alpha, img, 1-alpha, 0 )
cv2.imshow('image',overlay)

In [ ]:
img = imgs[0].copy()
im3 = img.copy()
alpha = 0.5
cv2.rectangle(im3, (33 ,28),(496 ,494),(0, 0, 255), -1)
cv2.addWeighted(im3, alpha, img, 1 - alpha,0, img)
cv2.imshow("output", img)

In [ ]:
cv2.rectangle(img,(ix,iy),(xx,yy),(0,255,0),2)
im2 = img.copy()
io.imshow(im2)

In [ ]:
# get user input
for ii in range(4):
    a = input("behavior")
    print(a)


In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img,'frame:' + str(ctr),(1,30), font, 1,(255,255,255),2,cv2.LINE_AA)
cv2.imshow("img", img)

In [ ]:
# open dialog box

In [ ]:
import tkinter as tk
from tkinter import simpledialog
def center_window(root, width=300, height=300):
    # get screen width and height
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # calculate position x and y coordinates
    x = (screen_width/2) - (width/2)
    y = (screen_height/2) - (height/2)
    root.geometry('%dx%d+%d+%d' % (width, height, x, y))



application_window = tk.Tk()
center_window( application_window)


answer = simpledialog.askstring("Input", "What the behavior?",
                                parent=application_window)
if answer is not None:
    print("Your first name is ", answer)
else:
    print("You don't have a first name?")
application_window.destroy()



In [ ]:
print("hi")